In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)  # 显示所有行
import numpy as np
#import jieba
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold,train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2, RFE
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
import math
import pandas as pd 
import numpy as np
from pandas import Series
from sklearn import metrics
import pandas

import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np  
from random import randint
from sqlite3 import adapters

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import sklearn.metrics as metrics
from sklearn.impute import SimpleImputer  #导包



##  1.feature selection

In [ ]:
class FeatureSelector:
    def __init__(self, X, y, featurenum_criteria):
        self.X = X
        self.y = y
        self.featurenum_criteria = featurenum_criteria

    def pearson_corr(self):
        corr_scores = []
        for col in self.X.columns:
            corr_score, _ = pearsonr(self.X[col], self.y)
            corr_scores.append((col, abs(corr_score)))
        corr_scores.sort(key=lambda x: x[1], reverse=True)
        return [col for col, _ in corr_scores[:self.featurenum_criteria]]

    def chi_square(self):
        selector = SelectKBest(chi2, k=self.featurenum_criteria)
        selector.fit(self.X, self.y)
        return self.X.columns[selector.get_support(indices=True)].tolist()

    def RRE(self):
        model = GradientBoostingClassifier()  # You can use any model of your choice
        rfe = RFE(estimator=model, n_features_to_select=self.featurenum_criteria)
        rfe.fit(self.X, self.y)
        return self.X.columns[rfe.support_].tolist()

    def embedded_lgbm(self):
        model = GradientBoostingClassifier()
        model.fit(self.X, self.y)
        feature_importances = model.feature_importances_
        feature_importances = pd.Series(feature_importances, index=self.X.columns)
        feature_importances = feature_importances.sort_values(ascending=False)
        return feature_importances.index[:self.featurenum_criteria].tolist()

    def calculate_embedET(self, randomstate):
        model = ExtraTreesClassifier(n_estimators=5, criterion='gini', max_features=2, random_state=randomstate)
        model.fit(self.X, self.y)

        features = pd.DataFrame()
        features['names'] = self.X.columns
        features['coef'] = abs(model.feature_importances_)
        features = features.sort_values(by='coef', ascending=False)
        num = self.featurenum_criteria if self.featurenum_criteria > 1 else math.ceil(
            self.featurenum_criteria * (self.X.shape[0]))
        data_sorted = features.head(num)
        colnames_sorted = data_sorted['names'].values
        return colnames_sorted[:self.featurenum_criteria]  # Ensure the same number of features

    def calculate_embedRF(self, randomstate):
        rf = RandomForestClassifier(n_estimators=30, max_depth=3, random_state=randomstate)
        model = rf.fit(self.X, self.y)
        features = pd.DataFrame()
        features['names'] = self.X.columns
        features['coef'] = abs(model.feature_importances_)
        features = features.sort_values(by='coef', ascending=False)
        num = self.featurenum_criteria if self.featurenum_criteria > 1 else math.ceil(
            self.featurenum_criteria * (self.X.shape[0]))
        data_sorted = features.head(num)
        colnames_sorted = data_sorted['names'].values
        return colnames_sorted[:self.featurenum_criteria]  # Ensure the same number of features

    def integrated_feature_selection(self, random_state=None):
        if random_state:
            np.random.seed(random_state)

        pearson_selected = self.pearson_corr()
        chi_selected = self.chi_square()
        rre_selected = self.RRE()
        embedded_lgbm_selected = self.embedded_lgbm()
        embedET_selected = self.calculate_embedET(random_state)
        embedRF_selected = self.calculate_embedRF(random_state)

        # Ensure the same number of features for all methods
        all_selected = [
            pearson_selected[:self.featurenum_criteria],
            chi_selected[:self.featurenum_criteria],
            rre_selected[:self.featurenum_criteria],
            embedded_lgbm_selected[:self.featurenum_criteria],
            embedET_selected[:self.featurenum_criteria],
            embedRF_selected[:self.featurenum_criteria]
        ]

        # Voting mechanism
        feature_votes = {}
        for feature_list in all_selected:
            for feature in feature_list:
                if feature in feature_votes:
                    feature_votes[feature] += 1
                else:
                    feature_votes[feature] = 1
        sorted_votes = sorted(feature_votes.items(), key=lambda x: x[1], reverse=True)
        final_selected_features = [feature for feature, _ in sorted_votes[:self.featurenum_criteria]]

        return final_selected_features


## 2.Pseudo-lable with putback(SS-PP)

In [ ]:

class PseudoTotal():
    def __init__(self,max_iter,basic_model,n_top,n_bottom,featnum_criteria,feature_traNum,put_back,model_use_Augment,need_model,fp):
        self.max_iter=max_iter
        self.n_top=n_top
        self.n_bottom=n_bottom
        self.basic_model=basic_model    
        self.feature_traNum=feature_traNum
        self.featnum_criteria=featnum_criteria
        self.vote_criteria=2
        #self.fp=fp
        self.put_back=put_back
        self.model_use_Augment=model_use_Augment
        self.need_model=need_model
        self.fp=fp

    #step2 特征选择
    def feature_selection_func(self,X,y,vote_criteria):
        #print(y.tail(2))
        feature=feature_selection(X=X,y=y,featurenum_criteria=self.featnum_criteria,vote_criteria=vote_criteria).main()
        return feature
    
    def model_train(self,AugmentedX=None,AugmentedY =None,features=None,model=None,X_train=None,y_train=None):
        #model=self.basic_model
        if self.model_use_Augment==True: 
            cur_model=model.fit(AugmentedX[features],AugmentedY) 
        else: 
            cur_model=model.fit(X_train[features],y_train) 
        return(cur_model)   
    
    def model_assess(self,model,features,X_test,y_test):
        '''specificity:特异性'''
        pred_test = model.predict(X_test[features])
        proba_test = model.predict_proba(X_test[features])[:, 1]
        a = metrics.confusion_matrix(y_test, pred_test)
        accuracy=metrics.accuracy_score(y_test, pred_test)
        precision=metrics.precision_score(y_test, pred_test) 
        #AUC=metrics.roc_auc_score(y_test, proba_test)
        recall=metrics.recall_score(y_test, pred_test)
        F1 = metrics.f1_score(y_test, pred_test)
        specificity=a[0][0]/(a[0][0]+a[0][1])
        #specificity2=a[1][1]/(a[1][1]+a[1][0])这个就是召回率
        Kappa=metrics.cohen_kappa_score(y_test, pred_test)
        
        #assDaFrame=pd.DataFrame([[accuracy,precision,recall,specificity,AUC,F1,Kappa]]) #注意]有两个
        assDaFrame=pd.DataFrame([[accuracy,precision,recall,specificity,F1,Kappa]]) #注意]有两个
        return assDaFrame,proba_test
    
    def AugmentData(self,features,cur_model,X_unlabled,X_train,y_train,):
        unlable_copy_first=X_unlabled.copy(deep=True)
        unlable_copy=unlable_copy_first[features].reset_index(drop=True) 
        
        #print('---------unlable_copy------')
        #print(unlable_copy.tail(2))
        unlable_copy['unlable_copy_pre']=cur_model.predict(unlable_copy)
        unlable_copy['unlable_copy_preprob']=cur_model.predict_proba(unlable_copy.iloc[:,:unlable_copy.shape[1]-1])[:,1]
        unlable_copy['Pseudo_lable']=-1
        
        numtop=self.n_top if self.n_top>1 else self.n_top*(unlable_copy.shape[0])
        numdown=self.n_bottom if self.n_bottom>1 else self.n_bottom*(unlable_copy.shape[0])

        Pseudo_top=unlable_copy.sort_values(by='unlable_copy_preprob').head(numtop)
        Pseudo_bottom=unlable_copy.sort_values(by='unlable_copy_preprob',ascending=False).head(numdown)
        Pseudo_top['Pseudo_lable']=0
        Pseudo_bottom['Pseudo_lable']=1
        Pseudo=Pseudo_top.append(Pseudo_bottom)
        Pseudo_index=[x for x in Pseudo.index]
        #print(Pseudo_index)
        
        new_data=X_unlabled.iloc[Pseudo_index]
        Augmented_X= X_train.append(new_data)
        Augmented_Y = y_train.append(Pseudo['Pseudo_lable'])
        
        if self.put_back==True:
            return Augmented_X,Augmented_Y
        else:
            unlablecopy_sort=unlable_copy.sort_values(by='unlable_copy_preprob')
            median_last_line=unlablecopy_sort.shape[0]-numdown-1
            Pseudo_median=unlablecopy_sort.iloc[numtop:median_last_line,:]  
            Pseudo_median_index=list(Pseudo_median.index)
            X_unlabled_append= X_unlabled.append(new_data)
            X_unlabled=X_unlabled_append.drop_duplicates(subset=X_unlabled.columns, keep=False) #删除所有重复项，不保留
            X_unlabled = X_unlabled.reset_index(drop=True)  #否则报错索引超出列表
            #print(X_unlabled.shape[0])
            return Augmented_X,Augmented_Y,X_unlabled
   
   
    def main(self):
        data=pd.read_csv(self.fp,encoding='gbk')
        data.dropna(subset=data.columns.difference(['MMSESCORE']), inplace=True)
        labeled_data = data[data['MMSESCORE'].notnull()]  
        unlabeled_data = data[data['MMSESCORE'].isnull()] 
        X_unlabled=unlabeled_data

        print(labeled_data.shape,unlabeled_data.shape)
        print("标签均衡情况：",labeled_data['MMSESCORE'].value_counts())
        # 分离特征和目标列
        X= labeled_data.drop(columns=['MMSESCORE',"ID"])
        y = labeled_data['MMSESCORE']
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        total_assess = pd.DataFrame()
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            feature=FeatureSelector(X_train,y_train,self.feature_traNum).integrated_feature_selection(random_state=42)
            
            # supervised
            assess_total=pd.DataFrame()
            cur_model=self.model_train(AugmentedX=X_train,AugmentedY =y_train,features=feature,model=self.basic_model,X_train=X_train,y_train=y_train)
            assess_score,proba_test=self.model_assess(model=cur_model,features=feature,X_test=X_test,y_test=y_test)
            assess_total=assess_total.append(assess_score)

            if self.put_back==True:
                Augmented_X,Augmented_Y =self.AugmentData(features=feature,cur_model=cur_model,X_unlabled=X_unlabled,X_train=X_train,y_train=y_train)
            else:
                Augmented_X,Augmented_Y,X_unlabled =self.AugmentData(features=feature,cur_model=cur_model,X_unlabled=X_unlabled,X_train=X_train,y_train=y_train)
            
            # semi_pseudo


            iter=1 
            featureiter=1
            while iter<=self.max_iter:
                cur_model=self.model_train(AugmentedX=Augmented_X,AugmentedY =Augmented_Y,features=feature,model=cur_model,X_train=X_train,y_train=y_train)
                assess_score,proba_test=self.model_assess(model=cur_model,features=feature,X_test=X_test,y_test=y_test)
                if self.put_back==True:
                    Augmented_X,Augmented_Y =self.AugmentData(features=feature,cur_model=cur_model,X_unlabled=X_unlabled,X_train=X_train,y_train=y_train)
                else:
                    Augmented_X,Augmented_Y,X_unlabled =self.AugmentData(features=feature,cur_model=cur_model,X_unlabled=X_unlabled,X_train=X_train,y_train=y_train)
                
                assess_total=assess_total.append(assess_score)
                
                iter+=1
                featureiter+=1
            
            #columns=['accuracy','precision','recall','specificity','AUC','F1','Kappa']
            columns=['accuracy','precision','recall','specificity','F1','Kappa']
            assess_total.columns=columns
            print(assess_total)

            
            """
            if self.need_model==True:
                assess=[assess_total,y_test,proba_test]
                data=[X_train, X_test, y_train, y_test]
                dataAug=[Augmented_X,Augmented_Y]
                model=[feature,cur_model]
                #return assess_total,y_test,proba_test,feature,cur_model,X_train
                return assess,data,dataAug,model,feature
            else:
                print(assess_total)
                return assess_total"""
            total_assess=total_assess.append(assess_total)
    
        print('训练轮次+交叉验证：',total_assess)
        average_assess = total_assess.mean()
        print('五折平均', average_assess)
        return average_assess

    

In [ ]:
# Exemple
model = RandomForestClassifier(n_estimators=100, random_state=42)
a=PseudoTotal(max_iter=30,
            basic_model=model,
            n_top=10,n_bottom=10,
            featnum_criteria=35,
            feature_traNum=1,
            put_back=True,
            model_use_Augment=True,
            need_model=False,
            fp="9.dataset26criteria.csv").main()

## (1).base learners

In [ ]:
random_num=32
#基分类器  individual learner
classifier_bys = MultinomialNB()
classifier_svm = svm.SVC(random_state=random_num,max_iter=100000,probability=True)
classifier_mlp = MLPClassifier(random_state=random_num)
classifier_knn = KNeighborsClassifier(n_neighbors=10)
classifier_lrclf = LogisticRegression()

#集成学习
#boosting
classifier_ada = AdaBoostClassifier(random_state=random_num)
classifier_gbc = GradientBoostingClassifier(random_state=random_num)
classifier_xgb =XGBClassifier(random_state=random_num,eval_metric='mlogloss')
classifier_lgbm = lgb.LGBMClassifier(random_state=random_num)
#bagging
classifier_rf = RandomForestClassifier(random_state=random_num)
#stacking
classifier_scclf_individual= StackingCVClassifier(classifiers=[classifier_bys,classifier_svm,classifier_mlp ,classifier_knn ,classifier_lrclf], meta_classifier=classifier_lrclf,cv=5,random_state=random_num)
classifier_scclf_ensemble= StackingCVClassifier(classifiers=[classifier_ada,classifier_gbc,classifier_xgb ,classifier_lgbm,classifier_rf], meta_classifier=classifier_lrclf,cv=5,random_state=random_num)
#voting
voting_soft_individual= VotingClassifier(estimators=[
    ('classifier_bys', classifier_bys),
    ('classifier_svm', classifier_svm),
    ('classifier_mlp ', classifier_mlp ),
    ('classifier_lrclf',classifier_lrclf),
    ('classifier_knn' ,classifier_knn )
], voting='soft')
voting_soft_ensemble= VotingClassifier(estimators=[
    ('classifier_ada', classifier_ada),
    ('classifier_gbc', classifier_gbc),
    ('classifier_xgb ',classifier_xgb ),
    ('classifier_lgbm',classifier_lgbm),
    ('classifier_rf' ,classifier_rf )
], voting='soft')

In [ ]:
model_ensemble=[classifier_ada,classifier_gbc,classifier_xgb,classifier_lgbm,classifier_rf,classifier_scclf_ensemble,voting_soft_ensemble]
model_ensemble0=[classifier_ada,classifier_gbc,classifier_xgb,classifier_lgbm,classifier_rf]
model_ensemble1=[classifier_scclf_ensemble,voting_soft_ensemble]
model_ensemble2=[classifier_scclf_individual,voting_soft_individual]
model_individual=[classifier_bys,classifier_svm ,classifier_mlp,classifier_knn,classifier_lrclf]

In [ ]:
def tranin_with_feature(basic_model):
    asstotal=pd.DataFrame()
    for featurenum in range(20,101,5):
        #print(featurenum)            
        average_assess=PseudoTotal(max_iter=30,
            basic_model=model,
            n_top=10,
            n_bottom=10,
            featnum_criteria=featurenum,
            feature_traNum=1,
            put_back=True,
            model_use_Augment=True,
            need_model=False,
            fp="9.dataset26criteria.csv").main()
        assess['featurenum']=featurenum
        asstotal=asstotal.append(average_assess)
    return asstotal

## (2) supervised machine learning

In [ ]:
for model in model_ensemble:
    average_assess=PseudoTotal(max_iter=0,
            basic_model=model,
            n_top=10,
            n_bottom=10,
            featnum_criteria=featurenum,
            feature_traNum=1,
            put_back=True,
            model_use_Augment=True,
            need_model=False,
            fp="9.dataset26criteria.csv").main()
for model in model_individual:
    average_assess=PseudoTotal(max_iter=0,
            basic_model=model,
            n_top=10,
            n_bottom=10,
            featnum_criteria=featurenum,
            feature_traNum=1,
            put_back=True,
            model_use_Augment=True,
            need_model=False,
            fp="9.dataset26criteria.csv").main()

## (3) SS-PP

In [ ]:
FeaturePath=os.getcwd()+'\\Data\\feature'
for model in model_ensemble:
    assess=tranin_with_feature(model)
    patternloc=str(model).index('(')
    model_name=str(model)[0:patternloc].replace('Classifier',"").replace('CV',"")
    assess['model_name']=str(model_name)
    assess.to_csv(FeaturePath+f'\\featureselection_{model_name}.csv',index=False)
for model in model_ensemble2:
    assess=tranin_with_feature(model)
    patternloc=str(model).index('(')
    model_name=str(model)[0:patternloc].replace('Classifier',"").replace('CV',"")
    assess['model_name']='individual'+str(model_name)
    assess.to_csv(FeaturePath+f'\\featureselection_individual_{model_name}.csv',index=False)
for model in model_individual:
    assess=tranin_with_feature(model)
    patternloc=str(model).index('(')
    model_name=str(model)[0:patternloc].replace('Classifier',"").replace('CV',"")
    assess['model_name']=str(model_name)
    assess.to_csv(FeaturePath+f'\\featureselection_{model_name}.csv',index=False)

## 3.SELF-TRAINING without putback 

In [ ]:

assess=PseudoTotal(max_iter=30,
            basic_model=classifier_gbc ,
            n_top=10,n_bottom=10,
            featnum_criteria=35,
            feature_traNum=1,
            put_back=False,
            model_use_Augment=True,
            need_model=False,
            fp="9.dataset26criteria.csv").main()
assess

# 4.CO-TRANING

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.utils import shuffle

# 读取数据集
#data = pd.read_csv("7.ADMMSE.csv", encoding='gbk')
data = pd.read_csv("9.dataset26criteria.csv", encoding='gbk')
data.dropna(subset=data.columns.difference(['MMSESCORE']), inplace=True)
labeled_data = data[data['MMSESCORE'].notnull()]  
unlabeled_data = data[data['MMSESCORE'].isnull()] 

X_unlabeled = unlabeled_data.drop(columns=['MMSESCORE', 'ID'])
X_labeled = labeled_data.drop(columns=['MMSESCORE', 'ID'])
y_labeled = labeled_data['MMSESCORE']

def evaluate(y_true, y_pred, threshold=0.5):
    # 将连续目标值转换为二元值
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # 计算评价指标
    accuracy = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    roc_auc = roc_auc_score(y_true, y_pred)
    avg_precision = average_precision_score(y_true, y_pred)
    
    return accuracy, precision, recall, f1, roc_auc, avg_precision

# 定义随机森林分类器
rf_classifier = RandomForestClassifier()

# 定义交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初始化两个分类器
classifier1 = RandomForestClassifier(random_state=42)
classifier2 = classifier_gbc

# 定义迭代次数和停止条件
max_iterations = 5
iteration = 0
stop_condition = False


for train_index, test_index in kf.split(X_labeled):
    X_train, X_test = X_labeled.iloc[train_index], X_labeled.iloc[test_index]
    y_train, y_test = y_labeled.iloc[train_index], y_labeled.iloc[test_index]

    feature1=FeatureSelector(X_train,y_train,35).integrated_feature_selection(random_state=42)
    feature2=FeatureSelector(X_train,y_train,40).integrated_feature_selection(random_state=42)
    
    while iteration < max_iterations and not stop_condition:
        print(f"Iteration {iteration + 1}")

        # 每个分类器分别用已标记的数据进行训练
        classifier1.fit(X_train[feature1], y_train)
        classifier2.fit(X_train[feature2], y_train)

        # 使用两个分类器对未标记的数据进行预测
        y_pred1 = classifier1.predict(X_unlabeled)
        y_pred2 = classifier2.predict(X_unlabeled)

        # 选择每个分类器预测都一致的样本
        consistent_indices = (y_pred1 == y_pred2)
        consistent_X = X_unlabeled[consistent_indices]
        consistent_y = y_pred1[consistent_indices]

        # 将一致的样本添加到已标记的数据中
        X_train = pd.concat([X_train, consistent_X])
        y_train = np.concatenate([y_train, consistent_y])

        # 从未标记的数据中移除已经添加到已标记数据的样本
        X_unlabeled = X_unlabeled[~consistent_indices]

        # 如果未标记数据为空，停止迭代
        if len(X_unlabeled) == 0:
            stop_condition = True

        iteration += 1

    # 使用最终标记的数据重新训练分类器
    classifier1.fit(X_train, y_train)
    classifier2.fit(X_train, y_train)

    # 在测试集上评估性能f
    y_pred1 = classifier1.predict(X_test)
    y_pred2 = classifier2.predict(X_test)
    final_predictions = (y_pred1 + y_pred2) / 2  # 采用平均投票整合两个分类器的预测结果
    accuracy, precision, recall, f1, roc_auc, avg_precision = evaluate(y_test, final_predictions)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("ROC AUC Score:", roc_auc)
    print("Average Precision Score:", avg_precision)


# 5.Label-propogation

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.semi_supervised import LabelPropagation

# 读取数据集
data = pd.read_csv("9.dataset26criteria.csv", encoding='gbk')
data.dropna(subset=data.columns.difference(['MMSESCORE']), inplace=True)
labeled_data = data[data['MMSESCORE'].notnull()]  # 长度为189
unlabeled_data = data[data['MMSESCORE'].isnull()]  # 长度为189

X_unlabeled = unlabeled_data.drop(columns=['MMSESCORE', 'ID'])  # 长度为21w
X_labeled = labeled_data.drop(columns=['MMSESCORE', 'ID'])
y_labeled = labeled_data['MMSESCORE']

def evaluate(y_true, y_pred, threshold=0.5):
    # 将连续目标值转换为二元值
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # 计算评价指标
    accuracy = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    roc_auc = roc_auc_score(y_true, y_pred)
    avg_precision = average_precision_score(y_true, y_pred)
    
    return accuracy, precision, recall, f1, roc_auc, avg_precision

# 定义随机森林分类器
rf_classifier =  classifier_gbc

# 定义标签传播算法
lp_model = LabelPropagation()

# 将标签传播算法与随机森林分类器结合
X_combined = pd.concat([X_labeled, X_unlabeled])
y_combined = pd.concat([y_labeled, pd.Series([-1] * len(unlabeled_data))])  # 使用-1表示未标记数据

# 定义交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_combined):
    X_train, X_test = X_combined.iloc[train_index], X_combined.iloc[test_index]
    y_train, y_test = y_combined.iloc[train_index], y_combined.iloc[test_index]
    
    # 使用标签传播算法进行半监督学习，将未标记数据的标签传播给其它数据
    lp_model.fit(X_train, y_train)
    propagated_labels = lp_model.transduction_

    # 使用传播后的标签进行训练
    rf_classifier.fit(X_train, propagated_labels[train_index])

    # 在测试集上进行预测
    y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

    # 评估模型性能
    accuracy, precision, recall, f1, roc_auc, avg_precision = evaluate(y_test, y_pred_proba)
    print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}, ROC AUC: {:.4f}, Average Precision: {:.4f}"
          .format(accuracy, precision, recall, f1, roc_auc, avg_precision))


# 6.Graph semi-supevised

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Read the dataset
data = pd.read_csv("9.dataset26criteria.csv", encoding='gbk')

# Drop rows with missing values except for MMSESCORE column
data.dropna(subset=data.columns.difference(['MMSESCORE']), inplace=True)

# Separate labeled and unlabeled data
labeled_data = data[data['MMSESCORE'].notnull()]  
unlabeled_data = data[data['MMSESCORE'].isnull()]  

# Extract features and labels
X_unlabeled = unlabeled_data.drop(columns=['MMSESCORE', 'ID'])  
X_labeled = labeled_data.drop(columns=['MMSESCORE', 'ID'])
y_labeled = labeled_data['MMSESCORE']

# Function to evaluate model performance
def evaluate(y_true, y_pred, threshold=0.5):
    y_pred_binary = (y_pred >= threshold).astype(int)
    accuracy = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    roc_auc = roc_auc_score(y_true, y_pred)
    avg_precision = average_precision_score(y_true, y_pred)
    return accuracy, precision, recall, f1, roc_auc, avg_precision

# Define Graph Neural Network model
class GraphNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphNet, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        return F.log_softmax(x, dim=1)

# Convert data into PyTorch Geometric format
X_unlabeled_tensor = torch.tensor(X_unlabeled.values, dtype=torch.float)
X_labeled_tensor = torch.tensor(X_labeled.values, dtype=torch.float)
y_labeled_tensor = torch.tensor(y_labeled.values, dtype=torch.long)

# Create a fully connected graph (you may need to customize this)
edge_index = torch.tensor([[i, j] for i in range(len(X_unlabeled)) for j in range(len(X_unlabeled)) if i != j], dtype=torch.long).t().contiguous()

# Create PyTorch Geometric Data objects
unlabeled_data = Data(x=X_unlabeled_tensor, edge_index=edge_index)
labeled_data = Data(x=X_labeled_tensor, edge_index=edge_index, y=y_labeled_tensor)

# Initialize model parameters
input_dim = X_unlabeled.shape[1]
hidden_dim = 64
output_dim = 2  

# Initialize model
model = GraphNet(input_dim, hidden_dim, output_dim)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training
def train_model(model, labeled_data, unlabeled_data, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(labeled_data)
        loss = criterion(out, labeled_data.y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Train the model
train_model(model, labeled_data, unlabeled_data, optimizer, criterion)
